In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# AutoML SDK: AutoML image object detection model


## Installation

Install the latest (preview) version of AutoML SDK.

In [ ]:
! pip3 install -U google-cloud-automl --user

Install the Google *cloud-storage* library as well.

In [ ]:
! pip3 install google-cloud-storage

### Restart the Kernel

Once you've installed the AutoML SDK and Google *cloud-storage*, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os

if not os.getenv("AUTORUN"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### GPU run-time

*Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select* **Runtime > Change Runtime Type > GPU**

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the AutoML APIs and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) is already installed in AutoML Notebooks.

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Project ID

**If you don't know your project ID**, try to get your project ID using `gcloud` command by executing the second cell below.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for AutoML. We recommend when possible, to choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You cannot use a Multi-Regional Storage bucket for training with AutoML. Not all regions provide support for all AutoML services. For the latest support per region, see [Region support for AutoML services]()


In [ ]:
REGION = "us-central1"  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append onto the name of resources which will be created in this tutorial.


In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your GCP account

**If you are using AutoML Notebooks**, your environment is already
authenticated. Skip this step.

*Note: If you are on an AutoML notebook and run the cell, the cell knows to skip executing the authentication steps.*

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Vertex, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

    # Log in to your account on Google Cloud
    ! gcloud auth login

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

This tutorial is designed to use training data that is in a public Cloud Storage bucket and a local Cloud Storage bucket for your batch predictions. You may alternatively use your own training data that you have stored in a local Cloud Storage bucket.

Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets.

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al gs://$BUCKET_NAME

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

#### Import AutoML SDK

Import the AutoM SDK into our Python environment.

In [ ]:
import os
import sys
import time

from google.cloud import automl
from google.protobuf.json_format import MessageToJson

#### AutoML constants

Setup up the following constants for AutoML:

- `PARENT`: The AutoM location root path for dataset, model and endpoint resources.

In [ ]:
# AutoM location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

## Clients

The AutoML SDK works as a client/server model. On your side (the Python script) you will create a client that sends requests and receives responses from the server (AutoML).

You will use several clients in this tutorial, so set them all up upfront.


In [ ]:
def automl_client():
    return automl.AutoMlClient()


def prediction_client():
    return automl.PredictionServiceClient()


def operations_client():
    return automl.AutoMlClient()._transport.operations_client


clients = {}
clients["automl"] = automl_client()
clients["prediction"] = prediction_client()
clients["operations"] = operations_client()

for client in clients.items():
    print(client)

In [ ]:
IMPORT_FILE = "gs://cloud-ml-data/img/openimage/csv/salads_ml_use.csv"

In [ ]:
%%capture
! gsutil cp -r gs://cloud-ml-data/img/openimage/ gs://$BUCKET_NAME

In [ ]:
! gsutil ls gs://$BUCKET_NAME

In [ ]:
import tensorflow as tf

all_files_csv = ! gsutil cat $IMPORT_FILE
all_files_csv = [l.replace("cloud-ml-data/img", BUCKET_NAME) for l in all_files_csv]

IMPORT_FILE = "gs://" + BUCKET_NAME + "/openimage/salads_ml_use.csv"
with tf.io.gfile.GFile(IMPORT_FILE, "w") as f:
    for l in all_files_csv:
        f.write(l + "\n")

In [ ]:
! gsutil cat $IMPORT_FILE | head -n 10

*Example output*:
```
TEST,gs://migration-ucaip-trainingaip-20210301091741/openimage/103/279324025_3e74a32a84_o.jpg,Baked Goods,0.005743,0.084985,,,0.567511,0.735736,,
TEST,gs://migration-ucaip-trainingaip-20210301091741/openimage/103/279324025_3e74a32a84_o.jpg,Salad,0.402759,0.310473,,,1.000000,0.982695,,
TEST,gs://migration-ucaip-trainingaip-20210301091741/openimage/1064/3167707458_7b2eebed9e_o.jpg,Cheese,0.000000,0.000000,,,0.054865,0.480665,,
TEST,gs://migration-ucaip-trainingaip-20210301091741/openimage/1064/3167707458_7b2eebed9e_o.jpg,Cheese,0.041131,0.401678,,,0.318230,0.785916,,
TEST,gs://migration-ucaip-trainingaip-20210301091741/openimage/1064/3167707458_7b2eebed9e_o.jpg,Cheese,0.116263,0.065161,,,0.451528,0.286489,,
TEST,gs://migration-ucaip-trainingaip-20210301091741/openimage/1064/3167707458_7b2eebed9e_o.jpg,Cheese,0.557359,0.411551,,,0.988760,0.731613,,
TEST,gs://migration-ucaip-trainingaip-20210301091741/openimage/1064/3167707458_7b2eebed9e_o.jpg,Cheese,0.562206,0.059401,,,0.876467,0.260982,,
TEST,gs://migration-ucaip-trainingaip-20210301091741/openimage/1064/3167707458_7b2eebed9e_o.jpg,Cheese,0.567861,0.000161,,,0.699543,0.077502,,
TEST,gs://migration-ucaip-trainingaip-20210301091741/openimage/1064/3167707458_7b2eebed9e_o.jpg,Cheese,0.916052,0.085569,,,1.000000,0.348036,,
TEST,gs://migration-ucaip-trainingaip-20210301091741/openimage/1064/3167707458_7b2eebed9e_o.jpg,Salad,0.000000,0.000000,,,1.000000,1.000000,,
```


## Create a dataset

### [projects.locations.datasets.create](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.datasets/create)

#### Request

In [ ]:
dataset = {
    "display_name": "salads_20210301091741",
    "image_object_detection_dataset_metadata": {},
}

print(
    MessageToJson(
        automl.CreateDatasetRequest(parent=PARENT, dataset=dataset).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "dataset": {
    "displayName": "salads_20210301091741",
    "imageObjectDetectionDatasetMetadata": {}
  }
}
```


#### Call

In [ ]:
request = clients["automl"].create_dataset(parent=PARENT, dataset=dataset)

#### Response

In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/datasets/IOD6853960213125398528"
}
```


In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split("/")[-1]

print(dataset_id)

### [projects.locations.datasets.importData](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.datasets/importData)

#### Request

In [ ]:
input_config = {"gcs_source": {"input_uris": [IMPORT_FILE]}}

print(
    MessageToJson(
        automl.ImportDataRequest(name=dataset_id, input_config=input_config).__dict__[
            "_pb"
        ]
    )
)

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/datasets/IOD6853960213125398528",
  "inputConfig": {
    "gcsSource": {
      "inputUris": [
        "gs://migration-ucaip-trainingaip-20210301091741/openimage/salads_ml_use.csv"
      ]
    }
  }
}
```


#### Call

In [ ]:
request = clients["automl"].import_data(name=dataset_id, input_config=input_config)

#### Response

In [ ]:
result = request.result()

print(MessageToJson(result))

*Example output*:
```
{}
```


## Train a model

### [projects.locations.models.create](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models/create)

#### Request

In [ ]:
model = {
    "display_name": "salads_" + TIMESTAMP,
    "dataset_id": dataset_short_id,
    "image_object_detection_model_metadata": {"train_budget_milli_node_hours": 20000},
}

print(
    MessageToJson(automl.CreateModelRequest(parent=PARENT, model=model).__dict__["_pb"])
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "model": {
    "displayName": "salads_20210301091741",
    "datasetId": "IOD6853960213125398528",
    "imageObjectDetectionModelMetadata": {
      "trainBudgetMilliNodeHours": "20000"
    }
  }
}
```


#### Call

In [ ]:
request = clients["automl"].create_model(parent=PARENT, model=model)

#### Response

In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/IOD3797407498105782272"
}
```


In [ ]:
# The full unique ID for the training pipeline
model_id = result.name
# The short numeric ID for the training pipeline
model_short_id = model_id.split("/")[-1]

print(model_id)

## Evaluate the model

### [projects.locations.models.modelEvaluations.list](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models.modelEvaluations/list)

#### Call

In [ ]:
request = clients["automl"].list_model_evaluations(parent=model_id, filter="")

#### Response

In [ ]:
import json

model_evaluations = [json.loads(MessageToJson(me.__dict__["_pb"])) for me in request]
# The evaluation slice
evaluation_slice = request.model_evaluation[0].name

print(json.dumps(model_evaluations, indent=2))

*Example output*:
```
[
  {
    "name": "projects/116273516712/locations/us-central1/models/IOD3797407498105782272/modelEvaluations/794074077030707821",
    "annotationSpecId": "6622317852164620288",
    "createTime": "2021-03-01T10:51:08.382156Z",
    "evaluatedExampleCount": 18,
    "imageObjectDetectionEvaluationMetrics": {
      "evaluatedBoundingBoxCount": 96,
      "boundingBoxMetricsEntries": [
        {
          "iouThreshold": 0.1,
          "meanAveragePrecision": 0.48460323,
          "confidenceMetricsEntries": [
            {
              "confidenceThreshold": 3.6381647e-05,
              "recall": 0.65625,
              "precision": 0.26359832,
              "f1Score": 0.3761194
            },
            {
              "confidenceThreshold": 0.00059657835,
              "recall": 0.6458333,
              "precision": 0.2792793,
              "f1Score": 0.38993713
            },
            {
              "confidenceThreshold": 0.0012138822,
              "recall": 0.6354167,
              "precision": 0.2961165,
              "f1Score": 0.4039735
            },
            {
              "confidenceThreshold": 0.003875596,
              "recall": 0.625,
              "precision": 0.32967034,
              "f1Score": 0.4316547
            },
            {
              "confidenceThreshold": 0.0065186527,
              "recall": 0.6145833,
              "precision": 0.3597561,
              "f1Score": 0.45384616
            },
            {
              "confidenceThreshold": 0.008566886,
              "recall": 0.6041667,
              "precision": 0.36708862,
              "f1Score": 0.45669293
            },
            
            # REMOVED FOR BREVITY
            
            {
              "confidenceThreshold": 0.024422897,
              "recall": 0.1875,
              "precision": 0.06818182,
              "f1Score": 0.10000001
            },
            {
              "confidenceThreshold": 0.03706667,
              "recall": 0.125,
              "precision": 0.06896552,
              "f1Score": 0.08888888
            },
            {
              "confidenceThreshold": 0.17009574,
              "recall": 0.0625,
              "precision": 0.16666667,
              "f1Score": 0.09090909
            },
            {
              "confidenceThreshold": 0.9619299,
              "recall": 0.0625,
              "precision": 1.0,
              "f1Score": 0.11764706
            }
          ]
        },
        {
          "iouThreshold": 0.85,
          "confidenceMetricsEntries": [
            {
              "confidenceThreshold": 4.645326e-05
            },
            {
              "confidenceThreshold": 0.9619299
            }
          ]
        }
      ],
      "boundingBoxMeanAveragePrecision": 0.08613319
    },
    "displayName": "Baked Goods"
  }
]
```


### [projects.locations.models.modelEvaluations.get](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models.modelEvaluations/get)

#### Call

In [ ]:
request = clients["automl"].get_model_evaluation(name=evaluation_slice)

#### Response

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/IOD3797407498105782272/modelEvaluations/794074077030707821",
  "annotationSpecId": "6622317852164620288",
  "createTime": "2021-03-01T10:51:08.382156Z",
  "evaluatedExampleCount": 18,
  "imageObjectDetectionEvaluationMetrics": {
    "evaluatedBoundingBoxCount": 96,
    "boundingBoxMetricsEntries": [
      {
        "iouThreshold": 0.1,
        "meanAveragePrecision": 0.48460323,
        "confidenceMetricsEntries": [
          {
            "confidenceThreshold": 3.6381647e-05,
            "recall": 0.65625,
            "precision": 0.26359832,
            "f1Score": 0.3761194
          },
          {
            "confidenceThreshold": 0.00059657835,
            "recall": 0.6458333,
            "precision": 0.2792793,
            "f1Score": 0.38993713
          },
          
          # REMOVED FOR BREVITY
          
          {
            "confidenceThreshold": 0.9941245,
            "recall": 0.03125,
            "precision": 0.6,
            "f1Score": 0.05940594
          },
          {
            "confidenceThreshold": 0.99589527,
            "recall": 0.020833334,
            "precision": 0.6666667,
            "f1Score": 0.040404044
          },
          {
            "confidenceThreshold": 0.9964624,
            "recall": 0.010416667,
            "precision": 0.5,
            "f1Score": 0.020408163
          },
          {
            "confidenceThreshold": 0.9993537,
            "recall": 0.010416667,
            "precision": 1.0,
            "f1Score": 0.020618558
          }
        ]
      }
    ],
    "boundingBoxMeanAveragePrecision": 0.30185306
  },
  "displayName": "Tomato"
}
```


## Make batch predictions

### Make a batch prediction file

In [ ]:
import json

import tensorflow as tf

test_items = ! gsutil cat $IMPORT_FILE | head -n 10

gcs_input_uri = "gs://" + BUCKET_NAME + "/test.csv"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    for item in test_items:
        f.write(item.split(",")[1] + "\n")

! gsutil cat $gcs_input_uri

*Example output*:
```
gs://migration-ucaip-trainingaip-20210301091741/openimage/103/279324025_3e74a32a84_o.jpg
gs://migration-ucaip-trainingaip-20210301091741/openimage/103/279324025_3e74a32a84_o.jpg
gs://migration-ucaip-trainingaip-20210301091741/openimage/1064/3167707458_7b2eebed9e_o.jpg
gs://migration-ucaip-trainingaip-20210301091741/openimage/1064/3167707458_7b2eebed9e_o.jpg
gs://migration-ucaip-trainingaip-20210301091741/openimage/1064/3167707458_7b2eebed9e_o.jpg
gs://migration-ucaip-trainingaip-20210301091741/openimage/1064/3167707458_7b2eebed9e_o.jpg
gs://migration-ucaip-trainingaip-20210301091741/openimage/1064/3167707458_7b2eebed9e_o.jpg
gs://migration-ucaip-trainingaip-20210301091741/openimage/1064/3167707458_7b2eebed9e_o.jpg
gs://migration-ucaip-trainingaip-20210301091741/openimage/1064/3167707458_7b2eebed9e_o.jpg
gs://migration-ucaip-trainingaip-20210301091741/openimage/1064/3167707458_7b2eebed9e_o.jpg
```


### [projects.locations.models.batchPredict](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models/batchPredict)

#### Request

In [ ]:
input_config = {"gcs_source": {"input_uris": [gcs_input_uri]}}

output_config = {
    "gcs_destination": {"output_uri_prefix": "gs://" + f"{BUCKET_NAME}/batch_output/"}
}

print(
    MessageToJson(
        automl.BatchPredictRequest(
            name=model_id, input_config=input_config, output_config=output_config
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/IOD3797407498105782272",
  "inputConfig": {
    "gcsSource": {
      "inputUris": [
        "gs://migration-ucaip-trainingaip-20210301091741/test.csv"
      ]
    }
  },
  "outputConfig": {
    "gcsDestination": {
      "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210301091741/batch_output/"
    }
  }
}
```


#### Call

In [ ]:
request = clients["prediction"].batch_predict(
    name=model_id, input_config=input_config, output_config=output_config
)

#### Response

In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{}
```

In [ ]:
destination_uri = output_config["gcs_destination"]["output_uri_prefix"][:-1]

! gsutil ls $destination_uri/*
! gsutil cat $destination_uri/prediction*/*.jsonl

*Example output*:
```
gs://migration-ucaip-trainingaip-20210301091741/batch_output/prediction-salads_20210301091741-2021-03-01T10:52:53.972802Z/image_object_detection_0.jsonl
gs://migration-ucaip-trainingaip-20210301091741/batch_output/prediction-salads_20210301091741-2021-03-01T10:52:53.972802Z/image_object_detection_1.jsonl
gs://migration-ucaip-trainingaip-20210301091741/batch_output/prediction-salads_20210301091741-2021-03-01T10:52:53.972802Z/image_object_detection_2.jsonl
{"ID":"gs://migration-ucaip-trainingaip-20210301091741/openimage/103/279324025_3e74a32a84_o.jpg","annotations":[{"annotation_spec_id":"3163553338344079360","display_name":"Baked Goods","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.019032001,"y":0.047988813},{"x":0.5898183,"y":0.78811049}],"vertices":[]},"score":0.96192992}},{"annotation_spec_id":"6622317852164620288","display_name":"Tomato","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.55052197,"y":0.58303279},{"x":0.63364756,"y":0.74831831}],"vertices":[]},"score":0.870511}},{"annotation_spec_id":"3163553338344079360","display_name":"Baked Goods","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.056096584},{"x":0.91519189,"y":0.98487538}],"vertices":[]},"score":0.79013598}},{"annotation_spec_id":"4316474842950926336","display_name":"Salad","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.2624923,"y":0.063098952},{"x":0.99545622,"y":1}],"vertices":[]},"score":0.60728121}}]}
{"ID":"gs://migration-ucaip-trainingaip-20210301091741/openimage/103/279324025_3e74a32a84_o.jpg","annotations":[{"annotation_spec_id":"3163553338344079360","display_name":"Baked Goods","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.019032001,"y":0.047988813},{"x":0.5898183,"y":0.78811049}],"vertices":[]},"score":0.96192992}},{"annotation_spec_id":"6622317852164620288","display_name":"Tomato","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.55052197,"y":0.58303279},{"x":0.63364756,"y":0.74831831}],"vertices":[]},"score":0.870511}},{"annotation_spec_id":"3163553338344079360","display_name":"Baked Goods","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.056096584},{"x":0.91519189,"y":0.98487538}],"vertices":[]},"score":0.79013598}},{"annotation_spec_id":"4316474842950926336","display_name":"Salad","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.2624923,"y":0.063098952},{"x":0.99545622,"y":1}],"vertices":[]},"score":0.60728121}}]}
{"ID":"gs://migration-ucaip-trainingaip-20210301091741/openimage/1064/3167707458_7b2eebed9e_o.jpg","annotations":[{"annotation_spec_id":"2010631833737232384","display_name":"Cheese","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.5891307,"y":0.055182356},{"x":0.88701528,"y":0.2786459}],"vertices":[]},"score":0.99448556}},{"annotation_spec_id":"4316474842950926336","display_name":"Salad","image_object_detection":{"bounding_box":{"normalized_vertices":[{},{"x":0.99119592,"y":1}],"vertices":[]},"score":0.78258878}},{"annotation_spec_id":"2010631833737232384","display_name":"Cheese","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.09908694,"y":0.038005471},{"x":0.53745395,"y":0.41128135}],"vertices":[]},"score":0.61209571}},{"annotation_spec_id":"6622317852164620288","display_name":"Tomato","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.56387162,"y":0.40369061},{"x":0.99384451,"y":0.9047623299999999}],"vertices":[]},"score":0.57341981}},{"annotation_spec_id":"2010631833737232384","display_name":"Cheese","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.58524036,"y":0.41812626},{"x":0.99431562,"y":0.8581754}],"vertices":[]},"score":0.50971383}}]}
{"ID":"gs://migration-ucaip-trainingaip-20210301091741/openimage/1064/3167707458_7b2eebed9e_o.jpg","annotations":[{"annotation_spec_id":"2010631833737232384","display_name":"Cheese","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.5891307,"y":0.055182356},{"x":0.88701528,"y":0.2786459}],"vertices":[]},"score":0.99448556}},{"annotation_spec_id":"4316474842950926336","display_name":"Salad","image_object_detection":{"bounding_box":{"normalized_vertices":[{},{"x":0.99119592,"y":1}],"vertices":[]},"score":0.78258878}},{"annotation_spec_id":"2010631833737232384","display_name":"Cheese","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.09908694,"y":0.038005471},{"x":0.53745395,"y":0.41128135}],"vertices":[]},"score":0.61209571}},{"annotation_spec_id":"6622317852164620288","display_name":"Tomato","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.56387162,"y":0.40369061},{"x":0.99384451,"y":0.9047623299999999}],"vertices":[]},"score":0.57341981}},{"annotation_spec_id":"2010631833737232384","display_name":"Cheese","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.58524036,"y":0.41812626},{"x":0.99431562,"y":0.8581754}],"vertices":[]},"score":0.50971383}}]}
{"ID":"gs://migration-ucaip-trainingaip-20210301091741/openimage/1064/3167707458_7b2eebed9e_o.jpg","annotations":[{"annotation_spec_id":"2010631833737232384","display_name":"Cheese","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.5891307,"y":0.055182356},{"x":0.88701528,"y":0.2786459}],"vertices":[]},"score":0.99448556}},{"annotation_spec_id":"4316474842950926336","display_name":"Salad","image_object_detection":{"bounding_box":{"normalized_vertices":[{},{"x":0.99119592,"y":1}],"vertices":[]},"score":0.78258878}},{"annotation_spec_id":"2010631833737232384","display_name":"Cheese","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.09908694,"y":0.038005471},{"x":0.53745395,"y":0.41128135}],"vertices":[]},"score":0.61209571}},{"annotation_spec_id":"6622317852164620288","display_name":"Tomato","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.56387162,"y":0.40369061},{"x":0.99384451,"y":0.9047623299999999}],"vertices":[]},"score":0.57341981}},{"annotation_spec_id":"2010631833737232384","display_name":"Cheese","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.58524036,"y":0.41812626},{"x":0.99431562,"y":0.8581754}],"vertices":[]},"score":0.50971383}}]}
{"ID":"gs://migration-ucaip-trainingaip-20210301091741/openimage/1064/3167707458_7b2eebed9e_o.jpg","annotations":[{"annotation_spec_id":"2010631833737232384","display_name":"Cheese","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.5891307,"y":0.055182356},{"x":0.88701528,"y":0.2786459}],"vertices":[]},"score":0.99448556}},{"annotation_spec_id":"4316474842950926336","display_name":"Salad","image_object_detection":{"bounding_box":{"normalized_vertices":[{},{"x":0.99119592,"y":1}],"vertices":[]},"score":0.78258878}},{"annotation_spec_id":"2010631833737232384","display_name":"Cheese","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.09908694,"y":0.038005471},{"x":0.53745395,"y":0.41128135}],"vertices":[]},"score":0.61209571}},{"annotation_spec_id":"6622317852164620288","display_name":"Tomato","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.56387162,"y":0.40369061},{"x":0.99384451,"y":0.9047623299999999}],"vertices":[]},"score":0.57341981}},{"annotation_spec_id":"2010631833737232384","display_name":"Cheese","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.58524036,"y":0.41812626},{"x":0.99431562,"y":0.8581754}],"vertices":[]},"score":0.50971383}}]}
{"ID":"gs://migration-ucaip-trainingaip-20210301091741/openimage/1064/3167707458_7b2eebed9e_o.jpg","annotations":[{"annotation_spec_id":"2010631833737232384","display_name":"Cheese","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.5891307,"y":0.055182356},{"x":0.88701528,"y":0.2786459}],"vertices":[]},"score":0.99448556}},{"annotation_spec_id":"4316474842950926336","display_name":"Salad","image_object_detection":{"bounding_box":{"normalized_vertices":[{},{"x":0.99119592,"y":1}],"vertices":[]},"score":0.78258878}},{"annotation_spec_id":"2010631833737232384","display_name":"Cheese","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.09908694,"y":0.038005471},{"x":0.53745395,"y":0.41128135}],"vertices":[]},"score":0.61209571}},{"annotation_spec_id":"6622317852164620288","display_name":"Tomato","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.56387162,"y":0.40369061},{"x":0.99384451,"y":0.9047623299999999}],"vertices":[]},"score":0.57341981}},{"annotation_spec_id":"2010631833737232384","display_name":"Cheese","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.58524036,"y":0.41812626},{"x":0.99431562,"y":0.8581754}],"vertices":[]},"score":0.50971383}}]}
{"ID":"gs://migration-ucaip-trainingaip-20210301091741/openimage/1064/3167707458_7b2eebed9e_o.jpg","annotations":[{"annotation_spec_id":"2010631833737232384","display_name":"Cheese","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.5891307,"y":0.055182356},{"x":0.88701528,"y":0.2786459}],"vertices":[]},"score":0.99448556}},{"annotation_spec_id":"4316474842950926336","display_name":"Salad","image_object_detection":{"bounding_box":{"normalized_vertices":[{},{"x":0.99119592,"y":1}],"vertices":[]},"score":0.78258878}},{"annotation_spec_id":"2010631833737232384","display_name":"Cheese","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.09908694,"y":0.038005471},{"x":0.53745395,"y":0.41128135}],"vertices":[]},"score":0.61209571}},{"annotation_spec_id":"6622317852164620288","display_name":"Tomato","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.56387162,"y":0.40369061},{"x":0.99384451,"y":0.9047623299999999}],"vertices":[]},"score":0.57341981}},{"annotation_spec_id":"2010631833737232384","display_name":"Cheese","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.58524036,"y":0.41812626},{"x":0.99431562,"y":0.8581754}],"vertices":[]},"score":0.50971383}}]}
{"ID":"gs://migration-ucaip-trainingaip-20210301091741/openimage/1064/3167707458_7b2eebed9e_o.jpg","annotations":[{"annotation_spec_id":"2010631833737232384","display_name":"Cheese","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.5891307,"y":0.055182356},{"x":0.88701528,"y":0.2786459}],"vertices":[]},"score":0.99448556}},{"annotation_spec_id":"4316474842950926336","display_name":"Salad","image_object_detection":{"bounding_box":{"normalized_vertices":[{},{"x":0.99119592,"y":1}],"vertices":[]},"score":0.78258878}},{"annotation_spec_id":"2010631833737232384","display_name":"Cheese","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.09908694,"y":0.038005471},{"x":0.53745395,"y":0.41128135}],"vertices":[]},"score":0.61209571}},{"annotation_spec_id":"6622317852164620288","display_name":"Tomato","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.56387162,"y":0.40369061},{"x":0.99384451,"y":0.9047623299999999}],"vertices":[]},"score":0.57341981}},{"annotation_spec_id":"2010631833737232384","display_name":"Cheese","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.58524036,"y":0.41812626},{"x":0.99431562,"y":0.8581754}],"vertices":[]},"score":0.50971383}}]}
{"ID":"gs://migration-ucaip-trainingaip-20210301091741/openimage/1064/3167707458_7b2eebed9e_o.jpg","annotations":[{"annotation_spec_id":"2010631833737232384","display_name":"Cheese","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.5891307,"y":0.055182356},{"x":0.88701528,"y":0.2786459}],"vertices":[]},"score":0.99448556}},{"annotation_spec_id":"4316474842950926336","display_name":"Salad","image_object_detection":{"bounding_box":{"normalized_vertices":[{},{"x":0.99119592,"y":1}],"vertices":[]},"score":0.78258878}},{"annotation_spec_id":"2010631833737232384","display_name":"Cheese","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.09908694,"y":0.038005471},{"x":0.53745395,"y":0.41128135}],"vertices":[]},"score":0.61209571}},{"annotation_spec_id":"6622317852164620288","display_name":"Tomato","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.56387162,"y":0.40369061},{"x":0.99384451,"y":0.9047623299999999}],"vertices":[]},"score":0.57341981}},{"annotation_spec_id":"2010631833737232384","display_name":"Cheese","image_object_detection":{"bounding_box":{"normalized_vertices":[{"x":0.58524036,"y":0.41812626},{"x":0.99431562,"y":0.8581754}],"vertices":[]},"score":0.50971383}}]}
```


## Make online predictions

### Prepare file for online prediction

In [ ]:
import tensorflow as tf

test_item = ! gsutil cat $IMPORT_FILE | head -n 1
test_file = test_item[0].split(",")[1]

with tf.io.gfile.GFile(test_file, "rb") as f:
    content = f.read()

### [projects.locations.models.deploy](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models/deploy)

#### Call

In [ ]:
request = clients["automl"].deploy_model(name=model_id)

#### Response

In [ ]:
result = request.result()

print(MessageToJson(result))

*Example output*:
```
{}
```

### [projects.locations.models.predict](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models/predict)

#### Request

In [ ]:
image = automl.Image(image_bytes=content)
payload = automl.ExamplePayload(image=image)
params = {"score_threshold": "0.8"}

prediction_request = automl.PredictRequest(
    name=model_id, payload=payload, params=params
)

print(MessageToJson(prediction_request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/IOD3797407498105782272",
  "payload": {
    "image": {
      "imageBytes": "..."
    }
  },
  "params": {
    "score_threshold": "0.8"
  }
}
```


#### Call

In [ ]:
request = clients["prediction"].predict(request=prediction_request)

#### Response

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "payload": [
    {
      "annotationSpecId": "3163553338344079360",
      "imageObjectDetection": {
        "boundingBox": {
          "normalizedVertices": [
            {
              "x": 0.019032001,
              "y": 0.047988813
            },
            {
              "x": 0.5898183,
              "y": 0.7881106
            }
          ]
        },
        "score": 0.9619299
      },
      "displayName": "Baked Goods"
    },
    {
      "annotationSpecId": "6622317852164620288",
      "imageObjectDetection": {
        "boundingBox": {
          "normalizedVertices": [
            {
              "x": 0.55052197,
              "y": 0.5830328
            },
            {
              "x": 0.63364756,
              "y": 0.7483183
            }
          ]
        },
        "score": 0.8705111
      },
      "displayName": "Tomato"
    }
  ]
}
```


### [projects.locations.models.undeploy](https://cloud.google.com/automl/docs/reference/rest/v1/projects.locations.models/undeploy)

#### Request

In [ ]:
request = clients["automl"].undeploy_model(name=model_id)

#### Call

In [ ]:
result = request.result()

#### Response


In [ ]:
print(MessageToJson(result))

*Example output*:
```
{}
```

# Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

In [ ]:
delete_dataset = True
delete_model = True
delete_bucket = True

# Delete the dataset using the AutoML fully qualified identifier for the dataset
try:
    if delete_dataset:
        clients["automl"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the model using the AutoML fully qualified identifier for the model
try:
    if delete_model:
        clients["automl"].delete_model(name=model_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r gs://$BUCKET_NAME